In [1]:
# https://www.kaggle.com/shivamb/how-autoencoders-work-intro-and-usecases/data#2.1-UseCase-1-:-Image-Reconstruction
## load the libraries 
from keras.layers import Dense, Input, Conv2D, LSTM, MaxPool2D, UpSampling2D
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from numpy import argmax, array_equal
import matplotlib.pyplot as plt
from keras.models import Model
from imgaug import augmenters
from random import randint
import pandas as pd
import numpy as np

In [2]:
### read dataset 
train = pd.read_csv("data/fashion-mnist_train.csv")
train_x = train[list(train.columns)[1:]].values
train_y = train['label'].values

## normalize and reshape the predictors  
train_x = train_x / 255

## create train and validation datasets
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2)

## reshape the inputs
train_x = train_x.reshape(-1, 784)
val_x = val_x.reshape(-1, 784)

In [3]:
## input layer
input_layer = Input(shape=(784,))

## encoding architecture
encode_layer1 = Dense(1500, activation='relu')(input_layer)
encode_layer2 = Dense(1000, activation='relu')(encode_layer1)
encode_layer3 = Dense(500, activation='relu')(encode_layer2)

## latent view
latent_view   = Dense(10, activation='sigmoid')(encode_layer3)

## decoding architecture
decode_layer1 = Dense(500, activation='relu')(latent_view)
decode_layer2 = Dense(1000, activation='relu')(decode_layer1)
decode_layer3 = Dense(1500, activation='relu')(decode_layer2)

## output layer
output_layer  = Dense(784)(decode_layer3)

model = Model(input_layer, output_layer)

In [4]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
dense (Dense)                (None, 1500)              1177500   
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              1501000   
_________________________________________________________________
dense_2 (Dense)              (None, 500)               500500    
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5010      
_________________________________________________________________
dense_4 (Dense)              (None, 500)               5500      
_________________________________________________________________
dense_5 (Dense)              (None, 1000)             

In [5]:
shared_percent = 75 / 100
shared_train_data_size = int(len(train_x) * shared_percent)
num_models = 2

shared_train_data = train_x[:shared_train_data_size]
shared_train_labels = val_x[:shared_train_data_size]
private_train_data = np.split(train_x[shared_train_data_size:], num_models)
private_train_labels = np.split(
    val_x[shared_train_data_size:], num_models)

In [6]:
def save_weights(model, prefix):
    for i, layer in enumerate(model.layers):
        if len(layer.get_weights()) == 0:
            continue
        m = layer.get_weights()
        np.savetxt('weights/{}-w{}.out'.format(prefix, i), m[0], header="{},{}".format(*m[0].shape), delimiter=",")
        np.savetxt('weights/{}-b{}.out'.format(prefix, i), m[1], header="{},{}".format(m[1].shape, 1), delimiter=",")

In [7]:
og_file_name = 'models/og_{}_{}.h5'.format(int(shared_percent * 100), num_models)
model.save(og_file_name)

In [8]:
import keras
for i in range(num_models):
    print("Training model - {}".format(i))
    
    model_train_data = np.vstack((shared_train_data, private_train_data[i]))
    model_train_labels = np.vstack(
        (shared_train_labels, private_train_labels[i]))

    model = keras.models.load_model(og_file_name)
    
    model.compile(optimizer='adam', loss='mse')
    early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')

    model.fit(model_train_data, model_train_data, epochs=20, batch_size=2048, validation_data=(model_train_labels, model_train_labels), callbacks=[early_stopping])

    model_name = "models/clip_1_5_NO_{}_{}_{}.h5".format(i, int(shared_percent * 100), num_models)
    model.save(model_name)
    
    save_weights(model, "clip_1_5_NO_{}_{}_{}".format(i, int(shared_percent * 100), num_models))

Training model - 0
Epoch 1/20
21/21 [==============================] - 8s 401ms/step - loss: 0.0964 - val_loss: 0.0729
Epoch 2/20
21/21 [==============================] - 11s 504ms/step - loss: 0.0676 - val_loss: 0.0581
Epoch 3/20
21/21 [==============================] - 11s 508ms/step - loss: 0.0504 - val_loss: 0.0441
Epoch 4/20
21/21 [==============================] - 10s 498ms/step - loss: 0.0412 - val_loss: 0.0393
Epoch 5/20
21/21 [==============================] - 11s 512ms/step - loss: 0.0376 - val_loss: 0.0366
Epoch 6/20
21/21 [==============================] - 12s 583ms/step - loss: 0.0351 - val_loss: 0.0338
Epoch 7/20
21/21 [==============================] - 12s 559ms/step - loss: 0.0321 - val_loss: 0.0305
Epoch 8/20
21/21 [==============================] - 11s 512ms/step - loss: 0.0313 - val_loss: 0.0303
Epoch 9/20
21/21 [==============================] - 11s 541ms/step - loss: 0.0282 - val_loss: 0.0271
Epoch 10/20
21/21 [==============================] - 11s 547ms/step - los